In [1]:
# Suppress native-hadoop warning
!sed -i '$a\# Add the line for suppressing the NativeCodeLoader warning \nlog4j.logger.org.apache.hadoop.util.NativeCodeLoader=ERROR,console' /$HADOOP_HOME/etc/hadoop/log4j.properties

In [2]:
import sys
sys.path.append('/home/work')

BASE_DIR = '/home/work'

In [6]:
import pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.recommendation import ALSModel
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator

from data.utils.data_loader import load_from_hdfs

In [4]:
# Set Spark Settings
conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[*]'),
    ('spark.app.name', 'MusicRecommender'),
    # ('spark.executor.instances', '2'),  # Number of executors
    # ('spark.executor.cores', '8'),  # Cores per executor
    # ('spark.executor.memory', '10g'),  # Memory per executor
    ('spark.driver.memory','14g'),
])
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Print Spark Settings
settings = spark.sparkContext.getConf().getAll()
for s in settings:
    print(s)

('spark.app.submitTime', '1715811788579')
('spark.app.id', 'local-1715811789294')
('spark.executor.id', 'driver')
('spark.driver.host', '50278eaf4c0f')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false')
('spar

In [5]:
# Load Test Data
_, test_data = load_from_hdfs('raw', 1)

In [7]:
# Load Models
als_model = ALSModel.load(f'file://{BASE_DIR}/models/als_model')

# Define Evaluators
rmse = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# Make Predictions
predictions = als_model.transform(test_data)

# Evaluate Model
rmse_val = rmse.evaluate(predictions)

# Print Evaluation Metrics
print(f'RMSE: {rmse_val}')

RMSE: 1.154756499116459


In [10]:
# Show Predictions
predictions.show(20)

+-------+-------+------+------------+----------+
|user_id|song_id|rating|partition_id|prediction|
+-------+-------+------+------------+----------+
|      0|   7171|     5|           0|  4.207943|
|      0|   8637|     4|           0| 4.4320993|
|      0|  21966|     4|           0| 4.1567163|
|      0|  35821|     5|           0|  4.286071|
|      0|  82446|     5|           0| 4.6395755|
|      0|  90409|     5|           0|  4.163314|
|      0| 107410|     5|           0|  4.401118|
|      0| 131919|     5|           0| 4.8896813|
|      0| 132685|     3|           0| 4.2512097|
|      0| 136507|     3|           0| 4.1017985|
|      1|   3342|     5|           0| 3.9412968|
|      1|   7522|     1|           0| 2.1048512|
|      1|  25363|     2|           0| 3.2813597|
|      1|  38997|     5|           0| 3.6586933|
|      1|  43685|     1|           0| 1.5671213|
|      1|  45488|     4|           0| 3.7472024|
|      1|  62770|     3|           0| 3.1563296|
|      1| 109450|   